Copy of extract_FOV_and_cutout_galaxies, but for the resolution test.

In [1]:
import numpy as np
import eagle_constants_and_units as c
import cosmo_utils as csu
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits.axes_grid1 as axgrid
from astropy import constants as const
from astropy import units as u

from matplotlib.patches import ConnectionPatch

import os
import get_halpha_SB

%matplotlib inline

%run 'load_data.ipynb'
%run 'add_noise.ipynb'
%run 'extract_FOV.ipynb'
%run 'mask_galaxies.ipynb'

In [ ]:
## Loading all the files from the 25 Mpc box and adding them together

machine='coho'

if machine=='chinook':
    directory='/Users/lokhorst/Data/EAGLE/Niagaraoutput/queue'
elif machine=='coho':
    #directory='/Users/deblokhorst/Desktop/EAGLE'
    directory='/Users/deblokhorst/Documents/EAGLE'

def load25Mpc(fname,namestring,machine=='chinook'):
    if os.path.isfile(fname):
        print 'Loading the data: %s'%fname
        data = np.load(fname)['arr_0']
        return data
    elif machine=='chinook':
        print 'Adding together smaller data files...'
        filelist = glob(directory+namestring)
        print filelist
        data = np.load(filelist[0])['arr_0']
        for filename in filelist[1:]:
            print '...adding: '+filename
            data = np.log10(10**data + 10**(np.load(filename)['arr_0']))
        np.savez(fname,data)
        return data
    else:
        print fname+' does not exist.  Need to copy over/make on chinook where the slices are.'
    
    return None


halpha_25mpc_fname = directory+'/emission_halpha_L0025N0376_28_test2_SmAb_C2Sm_8000pix_1.250000slice_total_noSFR.npz'
namestring = '/emission_halpha_L0025N0376_28_test2_SmAb_C2Sm_8000pix_1.250000slice*noSFR.npz'
halpha_25mpc = load25Mpc(halpha_25mpc_fname,namestring,machine=='chinook')

o3_25mpc_fname = directory+'/emission_o3_L0025N0376_28_test2_SmAb_C2Sm_8000pix_1.250000slice_total_noSFR.npz'
namestring = '/emission_o3_L0025N0376_28_test2_SmAb_C2Sm_8000pix_1.250000slice*noSFR.npz'
o3_25mpc = load25Mpc(o3_25mpc_fname,namestring,machine=='chinook')



In [ ]:
def imreduce(img, factor, log=True, method = 'average'):
    """
    img: 2D image array
    factor: factor by which to reduce the number of array elements along each axis
    log: whether or not the array contains log data values
    """
    if log:
        inimg = 10**img
    else:
        inimg = img
    inshape = np.array(img.shape)

    if np.sum(inshape%factor) != 0:
        print('Output grid must have a integer number of cells: cannot reduce image pixels by a factor %i'%factor)
        return None
    inimg = np.array(np.split(inimg,inshape[0]/factor,axis=0))
    inimg = np.array(np.split(inimg,inshape[1]/factor,axis=-1))

    inimg = np.sum(inimg,axis=-1)
    inimg = np.sum(inimg,axis=-1)
    
    if method == 'average':
        inimg = inimg/np.float(factor**2)
        #outimg = np.average(inimg[])
    if log:
        inimg = np.log10(inimg)
    return inimg.T

## reduce the resolution

o3nonper = imreduce(o3nonper,2)
lyalpha_25mpc = imreduce(lyalpha_25mpc,2)
o3_25mpc = imreduce(o3_25mpc,2)
halpha_25mpc= imreduce(halpha_25mpc,2)

In [ ]:
verbose = True
import eagleSqlTools as sql
  
mySim = ('RefL0100N1504',100.)
con   = sql.connect("dlokhorst",password="mxdPB54Y")  
myQuery = "SELECT \
            SH.StarFormationRate as SFR, \
            SH.CentreOfPotential_x as sh_x,\
            SH.CentreOfPotential_y as sh_y,\
            SH.CentreOfPotential_z as sh_z,\
            SH.MassType_Star as mstar,\
            FOF.Group_R_Crit200 as R200,\
            FOF.Group_M_Crit200 as M200, \
            FOF.Group_R_TopHat200 as RTopHat, \
            FOF.Group_M_TopHat200 as MTopHat, \
            SH.HalfMassProjRad_Star, \
            SH.HalfMassProjRad_Gas, \
            FOF.GroupMass as fof_mass\
        FROM   \
            %s_Subhalo as SH, \
            %s_FOF as FOF  \
        WHERE   \
            SH.MassType_Star > 1.0e9     \
            and SH.SnapNum = 28 \
            and SH.CentreOfPotential_z >= 10 \
            and SH.CentreOfPotential_z <= 15 \
            and FOF.SnapNum = SH.SnapNum \
            and FOF.GroupID = SH.GroupID"%(mySim[0],mySim[0])

if verbose:
    print myQuery
    
myData = sql.execute_query(con,myQuery)

#sfr  = myData['SFR'][:]
# Location of the galaxy
xgal = myData['sh_x'][:]                  # cMpc
ygal = myData['sh_y'][:]                  # cMpc
zgal = myData['sh_z'][:]                  # cMpc
# Stellar Mass of the galaxy
mgal = myData['mstar'][:]                 # M_solar
# Effective Radii
rhgas = myData['HalfMassProjRad_Gas'][:]  # pkpc
rhstar= myData['HalfMassProjRad_Star'][:] # pkpc
# Virial Radii and Halo Masses
R200 = myData['R200'][:]                  # pkpc
M200 = myData['M200'][:]                  # M_solar
RTopHat = myData['RTopHat'][:]            # pkpc
MTopHat = myData['MTopHat'][:]            # M_solar

print 'DONE.'

In [ ]:
## After that scare of not being able to execute the SQL search, I'm going to save this info to a text file.
with open("extract_FOV_and_cutout_galaxies_25Mpc_SQL_galaxy_info.txt", "w") as text_file:
    text_file.write('## xgal,ygal,zgal,mgal,rhgas,rhstar,R200,M200,RTopHat,MTopHat \n')
    for ind in range(len(xgal)):
        text_file.write('%f,%f,%f,%f,%f,%f,%f,%f,%f,%f \n'%
                        (xgal[ind],ygal[ind],zgal[ind],mgal[ind],rhgas[ind],rhstar[ind],R200[ind],M200[ind],RTopHat[ind],MTopHat[ind]))

In [ ]:
verbose=True
xmin=0;xmax=25;ymin=0;ymax=25;zmin=10;zmax=15

Mpcperpix = 0.477/1000.*(1.+0.0235) * 6.4
        
# Condition (use bitwise "&" and not boolean "and")
condition_10andup_all = np.array(    (rhstar*5. > (Mpcperpix*1000.)) &
                                 (rhgas*5. > (Mpcperpix*1000.)) &
                                 (mgal > 10**10) 
                            )
condition_9to10_all = np.array(    (rhstar*5. > (Mpcperpix*1000.)) &
                               (rhgas*5. > (Mpcperpix*1000.)) &
                               (mgal > 10**9) & (mgal <= 10**10)
                          )

condition_9andup_all = np.array(    (rhstar*5. > (Mpcperpix*1000.)) &
                                 (rhgas*5. > (Mpcperpix*1000.)) &
                                 (mgal > 10**9) 
                            )
condition_10to11_all = np.array(    (rhstar*5. > (Mpcperpix*1000.)) &
                               (rhgas*5. > (Mpcperpix*1000.)) &
                               (mgal > 10**10) & (mgal <= 10**11)
                          )
condition_11andup = np.array( (mgal > 10**11) & (rhstar*5. > (Mpcperpix*1000.)) & 
                             (rhgas*5. > (Mpcperpix*1000.)) )
ii_11andup = np.array(np.where(condition_11andup)[0]) 
ii_10andup_all = np.array(np.where(condition_10andup_all)[0]) 
ii_9andup_all = np.array(np.where(condition_9andup_all)[0]) 
ii_9to10_all  =np.array(np.where(condition_9to10_all)[0])
ii_10to11_all =np.array(np.where(condition_10to11_all)[0])

xgal_10andup_all      = np.array(xgal[ii_10andup_all])
ygal_10andup_all      = np.array(ygal[ii_10andup_all])
mgal_10andup_all      = np.array(mgal[ii_10andup_all])
rhstar_10andup_all    = np.array(rhstar[ii_10andup_all])
print len(ygal_10andup_all )
R200_10andup_all      = np.array(R200[ii_10andup_all])
M200_10andup_all      = np.array(M200[ii_10andup_all])
RTopHat_10andup_all   = np.array(RTopHat[ii_10andup_all])
MTopHat_10andup_all   = np.array(MTopHat[ii_10andup_all])

xgal_9andup_all      = np.array(xgal[ii_9andup_all])
ygal_9andup_all      = np.array(ygal[ii_9andup_all])
mgal_9andup_all      = np.array(mgal[ii_9andup_all])
rhstar_9andup_all    = np.array(rhstar[ii_9andup_all])
print len(ygal_9andup_all )
R200_9andup_all      = np.array(R200[ii_9andup_all])
M200_9andup_all      = np.array(M200[ii_9andup_all])
RTopHat_9andup_all   = np.array(RTopHat[ii_9andup_all])
MTopHat_9andup_all   = np.array(MTopHat[ii_9andup_all])

xgal_11andup_all     = np.array(xgal[ii_11andup])
ygal_11andup_all     = np.array(ygal[ii_11andup])
mgal_11andup_all     = np.array(mgal[ii_11andup])
rhstar_11andup_all   = np.array(rhstar[ii_11andup])
print len(xgal_11andup_all)
R200_11andup_all      = np.array(R200[ii_11andup])
M200_11andup_all      = np.array(M200[ii_11andup])
RTopHat_11andup_all   = np.array(RTopHat[ii_11andup])
MTopHat_11andup_all   = np.array(MTopHat[ii_11andup])

xgal_9to10_all      = np.array(xgal[ii_9to10_all])
ygal_9to10_all      = np.array(ygal[ii_9to10_all])
mgal_9to10_all      = np.array(mgal[ii_9to10_all])
rhstar_9to10_all    = np.array(rhstar[ii_9to10_all])
print len(ygal_9to10_all )
R200_9to10_all      = np.array(R200[ii_9to10_all])
M200_9to10_all      = np.array(M200[ii_9to10_all])
RTopHat_9to10_all   = np.array(RTopHat[ii_9to10_all])
MTopHat_9to10_all   = np.array(MTopHat[ii_9to10_all])

xgal_10to11_all      = np.array(xgal[ii_10to11_all])
ygal_10to11_all      = np.array(ygal[ii_10to11_all])
mgal_10to11_all      = np.array(mgal[ii_10to11_all])
rhstar_10to11_all    = np.array(rhstar[ii_10to11_all])
print len(ygal_10to11_all )
R200_10to11_all      = np.array(R200[ii_10to11_all])
M200_10to11_all      = np.array(M200[ii_10to11_all])
RTopHat_10to11_all   = np.array(RTopHat[ii_10to11_all])
MTopHat_10to11_all   = np.array(MTopHat[ii_10to11_all])

print len(ygal_9to10_all )+len(ygal_10to11_all )+ len(xgal_11andup_all)

In [ ]:
def radial_profile(data, center):
    data = 10**data
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)

    tbin = np.bincount(r.ravel(), data.ravel())  ## radial profile before dividing by num of data points
    nr = np.bincount(r.ravel())                  ## num of data points
    radialprofile = tbin / nr
    return np.log10(radialprofile )

Mpcperpix = 0.477/1000.*(1.+0.0235) * resolution         # at 100 Mpc from us
kpcperpix = 0.477/1000.*(1.+0.0235) * resolution * 1000. # at 100 Mpc from us


In [ ]:
tenandup  = False
nineandup = True
boxsize = 0.2

def Halphaprofileplot(data,xgal,ygal,boxsize,rhstar=None,linewidth=1,color='black',ax1=None,label='median',method='median',title = ''):
    """
    Input:  data:  Full simulation data (100Mpc box size) from which will measure the radial Halpha profiles
            xgal,ygal: Locations of galaxies from the galaxy catalogue
    """
    if ax1 is None:
        f, (ax1) = plt.subplots(1, 1,figsize=(5,5))
        
    data = data_tuple[0]; xsize = data.shape[0]; ysize = data.shape[1]

    azprofiles=[]; length = len(ygal); title=title
    ax1.set_title(title)
    radius = np.arange(46) * kpcperpix

    for j in range(length):
        minx,miny,maxx,maxy = xgal[j]-boxsize/2.,ygal[j]-boxsize/2.,xgal[j]+boxsize/2.,ygal[j]+boxsize/2.
        xystarts_tiny=[minx,miny]; size_tiny=[boxsize,boxsize] 
        data_tiny=data[minx*xsize/100.:maxx*xsize/100.,miny*ysize/100.:maxy*ysize/100.]
        center, radi = (data_tiny.shape[0]/2,data_tiny.shape[1]/2), 55
        rad = radial_profile(data_tiny, center)
        if len(rad)==46:
            azprofiles.append(rad)
            if method=='none':
                ax1.plot(radius,rad)
            else:
                ax1.plot(radius,rad,color='lightgrey',zorder=1,alpha=0.5)
            if rhstar is not None:
                ax1.plot([rhstar[j],rhstar[j]],[-4,8],color='peachpuff',zorder=1,alpha=0.3)
        else:
            'catch edge cases'
            print "The length of the profile (x,y=%s,%s) is not correct, length: %s"%\
                        (minx+boxsize/2,miny+boxsize/2,len(rad))

    azprofiles = np.array(azprofiles)
    
    ax1.set_ylim(-4,8)
    ax1.set_xlabel('radius [kpc]')
    
    if method=='average':
        meanprofile = np.log10(np.mean(10**(azprofiles), axis=0))
        sigmaprofile = np.log10(np.std(10**(azprofiles), axis=0))
        lowerr  = np.log10( 10**meanprofile - (10**sigmaprofile) ) 
        higherr = np.log10( 10**meanprofile + (10**sigmaprofile) ) 
        ax1.plot(radius,lowerr,color=color,linestyle='--')
        ax1.plot(radius,higherr,color=color,linestyle='--')
        ax1.plot(radius,meanprofile,color=color,linestyle='-') 
        return meanprofile,lowerr,higherr
    if method=='median':
        medianprofile = np.log10(np.median(10**(azprofiles), axis=0))
        p25profile = np.percentile(azprofiles, 25, axis=0)
        p75profile = np.percentile(azprofiles, 75, axis=0)
        ax1.plot(radius,medianprofile,color=color,linestyle='-',label=label,zorder=3,linewidth=linewidth) 
      #  ax1.plot(radius,p25profile,color=color,linestyle='--',label='25 percentile') 
      #  ax1.plot(radius,p75profile,color=color,linestyle='--',label='75 percentile') 
        ax1.fill_between(radius,p25profile, p75profile, facecolor =color,alpha=0.2,zorder=4)
        if rhstar is not None:
            ax1.plot([np.median(rhstar),np.median(rhstar)],[-4,8],color='orange',linewidth=2,zorder=2,alpha=0.5)
        return medianprofile,p25profile,p75profile

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 17

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
linewidth=2
f, (ax1,ax2,ax3) = plt.subplots(1, 3,figsize=(16,6))

median_9to10,p25_9to10,p75_9to10=Halphaprofileplot(data,xgal_9to10_all,ygal_9to10_all,boxsize,ax1=ax1,\
                                                      color='green',label='median',linewidth=linewidth)

#plotindividualprofiles(data,xgal_9andup,ygal_9andup,boxsize,ax1=ax1,only9=True,linewidth=linewidth)

median_10to11,p25_10to11,p75_10to11=Halphaprofileplot(data,xgal_10to11_all,ygal_10to11_all,boxsize,ax1=ax2,\
                                                         color='blue',label = 'median',linewidth=linewidth)
#plotindividualprofiles(data,xgal_9andup,ygal_9andup,boxsize,ax1=ax2,only10=True,linewidth=linewidth)

median_11andup,p25_11andup,p75_11andup=Halphaprofileplot(data,xgal_11andup_all,ygal_11andup_all,boxsize,ax1=ax3,\
                                                         color='orange',label = 'median',linewidth=linewidth)


reff_lw = 1.5
print 'The inner CGM for the first plot is: %s'%np.median(rhstar_9to10_all)
print 'The inner CGM for the second plot is: %s'%np.median(rhstar_10to11_all)
print 'The inner CGM for the third plot is: %s'%np.median(rhstar_11andup_all)
ax1.plot([np.median(rhstar_9to10_all),np.median(rhstar_9to10_all)],[-4,8],color='black',linewidth=reff_lw,zorder=2,alpha=0.5)
ax2.plot([np.median(rhstar_10to11_all),np.median(rhstar_10to11_all)],[-4,8],color='black',linewidth=reff_lw,zorder=2,alpha=0.5)
ax3.plot([np.median(rhstar_11andup_all),np.median(rhstar_11andup_all)],[-4,8],color='black',linewidth=reff_lw,zorder=2,alpha=0.5)

ax1.plot([5*np.median(rhstar_9to10_all),5*np.median(rhstar_9to10_all)],[-4,8],color='red',linewidth=reff_lw,zorder=2,alpha=0.5)
ax2.plot([5*np.median(rhstar_10to11_all),5*np.median(rhstar_10to11_all)],[-4,8],color='red',linewidth=reff_lw,zorder=2,alpha=0.5)
ax3.plot([5*np.median(rhstar_11andup_all),5*np.median(rhstar_11andup_all)],[-4,8],color='red',linewidth=reff_lw,zorder=2,alpha=0.5)

print 5*np.median(rhstar_9to10_all)
print 5*np.median(rhstar_10to11_all)
print 5*np.median(rhstar_11andup_all)

title1 = r'10$^9$ M$_{\odot}$ < M$_{\star}$ < 10$^{10}$ M$_{\odot}$'
title2 = r'10$^{10}$ M$_{\odot}$ < M$_{\star}$ < 10$^{11}$ M$_{\odot}$'
title3 = r'M$_{\star}$ > 10$^{11}$ M$_{\odot}$'

for ax,title in zip([ax1,ax2,ax3],[title1,title2,title3]):
    ax.grid(linestyle=':')
    ax.legend(loc=1, framealpha=0.5)
    ax.set_xlim(2.,140)
    ax.text(130,-3.5,title,horizontalalignment='right',fontsize=16)
for ax in [ax2,ax3]:
    ax.set_yticklabels([])
    ax.set_ylabel('')

ax1.set_ylabel(r'H$\alpha$ SB [log (ph cm$^{-2}$ s$^{-1}$ sr$^{-1}$)]')

ax5 = ax1.twiny()
ax6 = ax2.twiny()
ax7 = ax3.twiny()

loglog = True

if loglog:
    for ax in [ax1,ax2,ax3,ax5,ax6,ax7]:
    ax.set_xscale('log')

for ax,R200 in zip([ax5,ax6,ax7],[R200_9to10_all,R200_10to11_all,R200_11andup_all]):
    ax.set_xlabel('R/R$_{200}$')
    ax.set_xlim(2./np.median(R200),140./np.median(R200))

if loglog:
    ax5.set_xticks([0.05,0.1,0.5])
    ax5.set_xticklabels([0.05,0.1,0.5])
    ax6.set_xticks([0.01,0.05,0.1,0.5])
    ax6.set_xticklabels([0.01,0.05,0.1,0.5])
    ax7.set_xticks([0.01,0.05,0.1])
    ax7.set_xticklabels([0.01,0.05,0.1])
    for ax in [ax1,ax2,ax3]:
        ax.set_xlabel(r'radius (kpc)')
    for ax in [ax5,ax6,ax7]:
        ax.set_xlabel(r'R/R$_{200}$')
        
print np.median(R200_9to10_all)
print np.median(RTopHat_9to10_all)

print np.median(R200_10to11_all)
print np.median(RTopHat_10to11_all)

print np.median(R200_11andup_all)
print np.median(RTopHat_11andup_all)

        
#SB_cgs = 10**-22 *u.erg / u.arcsec**2 / u.cm**2 / u.s
def getSB_ph(SB_cgs,photon_wavelength=656.3e-9*u.m):
    return SB_cgs * photon_wavelength/(const.h.to('erg s') * const.c.to('m/s')) * (206265.*u.arcsec)**2/u.sr
def getSB_cgs(SB_ph,photon_wavelength=656.3e-9*u.m):
    return SB_ph / (photon_wavelength/(const.h.to('erg s') * const.c.to('m/s')) * (206265.*u.arcsec)**2/u.sr)
print getSB_ph(10**-19 *u.erg / u.arcsec**2 / u.cm**2 / u.s)

ax8 = ax3.twinx()
ax8.set_ylabel(r'H$\alpha$ SB [log (erg cm$^{-2}$ s$^{-1}$ arcsec$^{-2}$)]')
ax8.set_ylim(np.log10(getSB_cgs(10**-4).value),np.log10(getSB_cgs(10**8).value))
print getSB_cgs(10**-4),getSB_cgs(10**8)

## Read in the surface brightness limits from a text file: They were calculated in SignalToNoise_azimuthalaveraging_limits
##ff = open('../SignalToNoise_azimuthalaveraging_limits_sigma1_5.txt','r')
#ff = open('../SignalToNoise_azimuthalaveraging_limits_sigma3.txt','r')
#readradius = []
#readSB = []
#for line in ff:
#    if line[0]!='#'and len(line)>5:
#        tmp = line.split()
#        readradius.append(tmp[0])
#        readSB.append(tmp[1])
#ff.close()
#readradius = np.array(readradius,'d')
#readSB = np.array(readSB,'d')
#
##ax1.plot(readradius,np.log10(readSB),'k--',alpha=0.5)
##ax2.plot(readradius,np.log10(readSB),'k--',alpha=0.5)
##ax3.plot(readradius,np.log10(readSB),'k--',alpha=0.5)##
#
#ax1.fill_between(readradius,np.log10(readSB),y2 = -4,alpha=0.1,zorder = 2)
#ax2.fill_between(readradius,np.log10(readSB),y2 = -4,alpha=0.1,zorder = 2)
#ax3.fill_between(readradius,np.log10(readSB),y2 = -4,alpha=0.1,zorder = 2)

fig.subplots_adjust( wspace = 0, top=.95)
plt.tight_layout()

if loglog:
    printlog = '_log'
else:
    printlog = ''
plt.savefig('extract_FOV_and_cutout_galaxies_25Mpc%s.png'%printlog)